# 02 Correct Network Errors with Project Cards

In [1]:
import os
import sys
import yaml
import pickle
import glob

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [136]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = os.path.join('C:\\' 'Users', 'david.ory', 'Documents', 'GitHub', 'travel-model-two-networks')
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_03')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = os.path.join('C:\\' 'Users', 'david.ory', 'Documents', 'GitHub', 'mtc_Lasso')

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_03')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-12-04 09:14:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-04 09:14:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


## Read Version 00 Scenario

In [6]:
version_00_pickle_file_name = os.path.join(input_dir, 'working_scenario_00.pickle')
v_00_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

In [7]:
v_00_scenario.road_net.links_df.shape

(1634769, 31)

In [8]:
v_00_scenario.transit_net.feed.routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Data columns (total 16 columns):
route_id_original         701 non-null object
agency_id                 526 non-null object
route_short_name          665 non-null object
route_long_name           679 non-null object
route_desc                14 non-null object
route_type                701 non-null int64
route_url                 217 non-null object
route_color               260 non-null object
route_text_color          243 non-null object
agency_raw_name           701 non-null object
route_sort_order          95 non-null object
min_headway_minutes       32 non-null object
eligibility_restricted    4 non-null object
continuous_pickup         4 non-null object
continuous_drop_off       4 non-null object
route_id                  701 non-null object
dtypes: int64(1), object(15)
memory usage: 87.8+ KB


## Create Scenario and Apply Project Cards

In [9]:
v_01_scenario = Scenario.create_scenario(
    base_scenario=v_00_scenario,
    card_directory = card_dir,
    validate_project_cards=False)

2020-12-04 09:15:43, INFO: Creating Scenario
2020-12-04 09:15:43, INFO: Creating Scenario


In [10]:
v_01_scenario.apply_all_projects()

2020-12-04 09:15:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-12-04 09:15:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-12-04 09:15:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-12-04 09:15:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-12-04 09:15:45, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-12-04 09:15:45, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2020-12-04 09:15:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-04 09:15:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-04 09:15:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-04 09:15:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-04 09:15:45, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-04 09:15:45, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-04 09:15:46, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-12-04 09:15:46, INFO: Applying Year 2015 Modify HOV L

2020-12-04 09:16:57, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2020-12-04 09:16:57, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2020-12-04 09:16:57, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2020-12-04 09:17:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on Lawrence Expressway SB US 101 to I-280
2020-12-04 09:17:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on Lawrence Expressway SB US 101 to I-280
2020-12-04 09:17:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on Lawrence Expressway SB US 101 to I-280
2020-12-04 09:17:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on Lawrence Expressway SB US 101 to I-280
2020-12-04 09:17:16, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Lawrence Expressway SB US 101 to I-280
2020-12-0

2020-12-04 09:18:08, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-04 09:18:08, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-04 09:18:08, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-04 09:18:08, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-04 09:18:08, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-04 09:18:08, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-04 09:18:12, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from Bay Bridge to Carquinez Bridge Toll Plaza
2020-12-04 09:18:12, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from Bay Bridge to Carquinez Bridge Toll Plaza
2020-12-04 09:18:12, INFO: Applying Year 2015 Modi

2020-12-04 09:18:43, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV lanes and USE on Hwy 4 WB from Deer Valley Road to Port Chicago Highway
2020-12-04 09:18:48, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-04 09:18:48, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-04 09:18:48, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-04 09:18:48, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-04 09:18:48, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-04 09:18:48, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-04 09

2020-12-04 09:19:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-04 09:19:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-04 09:19:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-04 09:19:45, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-04 09:19:45, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-04 09:19:45, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-04 09:19:47, INFO: Applying Year 2015 Modify Express Lanes and USE on CA-237 WB I-880 to CA-262 Mission Blvd
2020-12-04 09:19:47, INFO: Applying Year 2015 Modify Express Lanes and USE on CA-237 WB I-880 to CA-262 Mission Blvd
2020-12-04 09:19:47, INFO: Applying Year 201

2020-12-04 09:19:53, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2020-12-04 09:19:53, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2020-12-04 09:19:53, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2020-12-04 09:19:53, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2020-12-04 09:19:53, INFO: Applying Year 2015 Bus Only Segments for Transbay Terminal Ramps
2020-12-04 09:19:53, INFO: Applying Year 2015 Bus Only Segments for Transbay Terminal Ramps
2020-12-04 09:19:53, INFO: Applying Year 2015 Bus Only Segments for Transbay Terminal Ramps
2020-12-04 09:19:53, INFO: Applying Year 2015 Bus Only Segments for Transbay Terminal Ramps
2020-12-04 09:19:53, INFO: Applying Project to Roadway Network: Year 2015 Bus Only Segments for Transbay Terminal Ramps
2020-12-04 09:19:53, INFO: Applying Project to Roadway Network: Year 2015 Bus Only S

In [11]:
v_01_scenario.applied_projects

['Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass',
 'Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass',
 'Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify HOV Lanes and USE on US 101 SB CA-85 to Oregon Expressway',
 'Year 2015 Modify HOV Lanes and USE on US 101 SB CA-85 to Cochrane Road',
 'Year 2015 Modify HOV Lanes and USE on US-101 NB (Segment 02) from North of Petaluma to South of Healdsburg',
 'Year 2015 Modify HOV Lanes and USE on US 101 NB Oregon Expressway to Whipple Road',
 'Year 2015 Modify HOV Lanes and USE on US-101 NB from North of Petaluma to South of Healdsburg',
 'Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85',
 'Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-8

In [12]:
v_01_scenario.road_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry', 'ft_cal', 'ft',
       'useclass', 'assignable', 'transit', 'managed', 'ML_lanes',
       'segment_id', 'tollbooth', 'ML_useclass', 'ML_access', 'ML_egress',
       'tollseg', 'tollboth', 'bus_only', 'transit_access'],
      dtype='object')

## Make Travel Model Networks

In [28]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_01_scenario.road_net, 
    parameters = parameters)

2020-12-04 09:37:57, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-04 09:37:57, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [29]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(1634769, 42)
(644475, 11)
(869970, 7)


In [30]:
model_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = model_net,
    parameters = parameters)

2020-12-04 09:38:16, INFO: Adding centroid and centroid connector to standard network
2020-12-04 09:38:16, INFO: Adding centroid and centroid connector to standard network
2020-12-04 09:38:16, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-04 09:38:16, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-04 09:38:27, INFO: Finished adding centroid and centroid connectors
2020-12-04 09:38:27, INFO: Finished adding centroid and centroid connectors


In [31]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2221035, 42)
(688952, 11)
(1163103, 7)


In [32]:
model_net.links_df.model_link_id.nunique()

2221035

In [33]:
model_net = mtc.add_tap_and_tap_connector(
    roadway_network = model_net,
    parameters = parameters)

2020-12-04 09:38:36, INFO: Adding tap and tap connector to standard network
2020-12-04 09:38:36, INFO: Adding tap and tap connector to standard network
2020-12-04 09:38:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-04 09:38:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-04 09:38:43, INFO: Finished adding tap and tap connectors
2020-12-04 09:38:43, INFO: Finished adding tap and tap connectors


In [34]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2264549, 42)
(694952, 11)
(1184860, 7)


In [35]:
model_net.links_df.model_link_id.nunique()

2264549

In [36]:
model_net.nodes_df.model_node_id.nunique()

694952

In [37]:
model_net.shapes_df.id.nunique()

1184860

In [38]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2020-12-04 09:39:18, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-12-04 09:39:18, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-12-04 09:39:18, INFO: Creating managed lane network.
2020-12-04 09:39:18, INFO: Creating managed lane network.
2020-12-04 09:39:18, INFO: Creating network with duplicated managed lanes
2020-12-04 09:39:18, INFO: Creating network with duplicated managed lanes


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\wang

2020-12-04 09:55:13, INFO: Determining cntype
2020-12-04 09:55:13, INFO: Determining cntype
2020-12-04 09:55:13, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-04 09:55:13, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-04 09:55:13, INFO: Calculating and adding roadway network variable: cntype
2020-12-04 09:55:13, INFO: Calculating and adding roadway network variable: cntype
2020-12-04 09:56:36, INFO: Finished determining variable: cntype
2020-12-04 09:56:36, INFO: Finished determining variable: cntype
2020-12-04 09:56:36, INFO: Determining transit
2020-12-04 09:56:36, INFO: Determining transit
2020-12-04 09:56:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-04 09:56:36, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-04 09:56:36, INFO: Variable 'transit' updated for some links. Returning without overwriting for those links. Calculating for other links
2

In [39]:
model_net.links_mtc_df.lanes_AM.value_counts()

1    2120565
2     104509
3      22883
4      13601
5       6393
6        822
7        127
0         37
8         18
9          1
Name: lanes_AM, dtype: int64

In [40]:
model_net.nodes_mtc_df.shape

(696464, 13)

In [45]:
model_net.nodes_mtc_df.N.nunique()

696460

In [43]:
model_net.links_mtc_df.shape

(2268956, 58)

In [44]:
model_net.links_mtc_df.model_link_id.nunique()

2268956

In [48]:
model_net.shapes_df.shape

(1189267, 8)

In [49]:
model_net.shapes_df.id.nunique()

1189267

## Write to Disk

## As shapefile

In [50]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2020-12-04 10:03:55, INFO: Writing Network as Shapefile
2020-12-04 10:03:55, INFO: Writing Network as Shapefile
2020-12-04 10:03:55, INFO: Renaming DBF Node Variables
2020-12-04 10:03:55, INFO: Renaming DBF Node Variables
2020-12-04 10:03:55, INFO: Renaming variables so that they are DBF-safe
2020-12-04 10:03:55, INFO: Renaming variables so that they are DBF-safe
2020-12-04 10:04:07, INFO: Renaming DBF Link Variables
2020-12-04 10:04:07, INFO: Renaming DBF Link Variables
2020-12-04 10:04:07, INFO: Renaming variables so that they are DBF-safe
2020-12-04 10:04:07, INFO: Renaming variables so that they are DBF-safe
2020-12-04 10:04:12, INFO: Writing Node Shapes:
 - Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks\data\processed\version_03\nodes.shp
2020-12-04 10:04:12, INFO: Writing Node Shapes:
 - Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks\data\processed\version_03\nodes.shp
2020-12-04 10:05:02, INFO: Writing Link Shapes:
 - Z:/Data/Users/Sijia/MTC/github/travel-mod

## As fixed width (for CUBE)

In [51]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2020-12-04 10:08:55, INFO: Starting fixed width convertion
2020-12-04 10:08:55, INFO: Starting fixed width convertion
2020-12-04 10:32:43, INFO: Writing out link database
2020-12-04 10:32:43, INFO: Writing out link database
2020-12-04 10:33:23, INFO: Writing out link header and width ----
2020-12-04 10:33:23, INFO: Writing out link header and width ----
2020-12-04 10:33:23, INFO: Starting fixed width convertion
2020-12-04 10:33:23, INFO: Starting fixed width convertion
2020-12-04 10:35:10, INFO: Writing out node database
2020-12-04 10:35:10, INFO: Writing out node database
2020-12-04 10:35:17, INFO: Writing out node header and width
2020-12-04 10:35:17, INFO: Writing out node header and width


In [18]:
%%capture
os.chdir(output_dir)
!make_cube.bat

In [19]:
%%capture
!runtpp make_simple_roadway_network.s
!runtpp make_complete_marin_network.s
!runtpp make_simple_transit_network.s

## As transit line file (for CUBE)

In [35]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_01_scenario.transit_net, parameters = parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2020-11-30 18:43:46, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-11-30 18:43:46, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-11-30 18:43:46, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2020-11-30 18:43:46, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
Empty DataFrame
Columns: [service_id, trip_headsign, direction_id, block_id, wheelchair_accessible, bikes_allowed, agency_raw_name, trip_short_name, original_trip_id, trip_bikes_allowed, trip_type, drt_max_travel_time, drt_avg_travel_time, drt_advance_book_min, drt_pickup_message, drt_drop_off_message, continuous_pickup_message, continuous_drop_off_message, route_id, trip_id, shape_id, route_id_original, agency_id, route_short_name, route_long_name, route_desc, route_type, route_url, route_color, route_text_color, route_sort_order, min_headway_minutes, eligibility_restricted, continuous_pickup, continuous_drop_off, headway_secs, start

## As pickle

In [21]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_01.pickle')
pickle.dump(v_01_scenario, open(working_scenario_filename, 'wb'))